# Necessary Imports and Extensions

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm import tqdm
import re
import collections
from wordcloud import STOPWORDS
from scipy.sparse import csr_matrix
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
import string
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from wordcloud import WordCloud
import gensim
import time
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import multiprocessing
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression


!pip install gensim
!pip install python-Levenshtein


[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading vader_lexicon: <urlopen error [WinError
[nltk_data]     10060] A connection attempt failed because the
[nltk_data]     connected party did not properly respond after a
[nltk_data]     period of time, or established connection failed
[nltk_data]     because connected host has failed to respond>


<br><br>

# From Here, Initiates the Training Phase of Model.

<strong><p style="color:blue;font-size:20px;"> Data Preprocessing is completed in the file named "Data Preprocessing.ipynb".</p>
   

In [28]:
train_data = pd.read_csv("preprocessed_tokenized_training_data.csv")
train_data['concatenated_description'] = train_data['concatenated_description'].map(str)

In [30]:
train_data['concatenated_description'] = train_data['concatenated_description'].apply(gensim.utils.simple_preprocess)

In [33]:
train_data

,Unnamed: 0,train_id,item_condition_id,price,shipping,concatenated_description
0,0,0,3,10.0,1,"[mlb, cincinnati, reds, shirt, size, xl, not, ..."
1,1,1,3,52.0,0,"[razer, blackwidow, chroma, keyboard, razer, k..."
2,2,2,1,10.0,1,"[avaviv, blouse, target, adorable, top, hint, ..."
3,3,3,1,35.0,1,"[leather, horse, statues, not, known, new, tag..."
4,4,4,1,44.0,0,"[gold, plated, rose, not, known, complete, cer..."
...,...,...,...,...,...,...
1481656,1481656,1482530,2,20.0,1,"[free, people, inspired, dress, free, people, ..."
1481657,1481657,1482531,2,14.0,0,"[little, mermaid, handmade, dress, disney, lit..."
1481658,1481658,1482532,2,12.0,0,"[day, fix, containers, eating, plan, not, know..."
1481659,1481659,1482533,3,45.0,1,"[world, markets, lanterns, not, known, one, se..."


<br><br>




<strong><p style="color:red;font-size:30px;">Run the next cell only if you want to train the model.</p><br><br>
<strong><p style="color:green;font-size:30px;">If already trained, Skip to the next cell and directly load the model.</p><br><br>
<strong><p style="color:blue;font-size:30px;">Note: I have trained the model and saved it as "word2vec_sg_model"</p>



In [13]:
cores = multiprocessing.cpu_count()

# Build the Word2Vec model
# Skip Gram
# Train the Word2Vec model

model1 = Word2Vec(vector_size=200, window=5, min_count=1, sg=1, workers=cores-1)
model1.build_vocab(train_data['concatenated_description'], progress_per=1000)
t = time.time()
model1.train(train_data['concatenated_description'], total_examples=model1.corpus_count, epochs=20)
print('Time to build vocab for Model-1: {} mins'.format(round((time.time() - t) / 60, 2)))
model1.save("word2vec_sg_model")

Time to build vocab for Model-1: 41.98 mins


<br><br>

<strong><p style="color:blue;font-size:30px;">Load the model.</p>


In [31]:
#Loading a pretrained model

model1 = Word2Vec.load("word2vec_sg_model")

In [32]:
def avg_word2vec1(doc):
    return np.mean([model1.wv[word] for word in doc if word in model1.wv.index_to_key], axis=0)

In [34]:
train_data_model1 = train_data.sample(frac=0.03, random_state=1)

series = train_data_model1.concatenated_description.apply(avg_word2vec1)

df = pd.DataFrame(series.apply(pd.Series))

train_data_model1 = pd.concat([train_data_model1, df], axis=1)

train_data_model1 = train_data_model1.drop(['concatenated_description'],axis=1)

In [5]:
train_data_model1

<br><br>

<strong><p style="color:blue;font-size:30px;">Saving the data set for future use.</p>

In [37]:
train_data_model1.to_csv('avgword2vec_sg.csv')

<br><br>

In [2]:
train_data_model1 = pd.read_csv('avgword2vec_sg.csv')

<br><br>



<strong><p style="color:green;font-size:30px;">"avgword2vec_sg.csv" Will be used for "train_test_split"</p>
    
 

# We've our dataset ready for the Model-1.

In [3]:
data = pd.read_csv('training_data_for_Model1.csv')

In [4]:
data.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
y = data['price']
X = data.drop(columns=['price'])

# Splitting the dataset

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
mlp = MLPRegressor(hidden_layer_sizes=5,max_iter=2000)
mlp.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=5, max_iter=2000)

In [8]:
y_pred = mlp.predict(X_test)

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
import numpy as np


# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

# R-squared (R²) or Coefficient of Determination
r2 = r2_score(y_test, y_pred)
print("R-squared (R²):", r2)

# Explained Variance Score
explained_var = explained_variance_score(y_test, y_pred)
print("Explained Variance Score:", explained_var)


Mean Squared Error (MSE): 1014.8595587715881
Root Mean Squared Error (RMSE): 31.85686046633579
Mean Absolute Error (MAE): 16.26224271749436
R-squared (R²): 0.2654078346837121
Explained Variance Score: 0.26621013883628675


In [11]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score

# Create an MLPRegressor model with desired hyperparameters
model = mlp

# Define the input features X and the target variable y

# Perform cross-validation with 5 folds
num_folds = 5
scores = cross_val_score(model, X, y, cv=num_folds, scoring='neg_mean_squared_error')

# Cross-validation returns negative mean squared error, so we take the absolute values
mse_scores = -scores

# Compute the mean and standard deviation of the cross-validation scores
mean_mse = mse_scores.mean()
std_mse = mse_scores.std()
# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_mse)
# Print the results
print("Mean MSE:", mean_mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Standard Deviation of MSE:", std_mse)

Mean MSE: 1228.9066361848895
Root Mean Squared Error (RMSE): 35.055764664101815
Standard Deviation of MSE: 301.8013828188631


In [12]:
import pickle
filename = 'sg_model1.pkl'
with open(filename, 'wb') as file:
    pickle.dump(mlp, file)
